In [1]:
import pandas as pd
import numpy as np
import geopandas
import geopy
import requests
from IPython.display import IFrame
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from sklearn.cluster import KMeans
import shapely.geometry
import pyproj
import csv
import shapely.geometry
import pyproj

In [2]:
#Strait of the real limit
Smin_lng = 2.225684
Smin_lat = 48.885438
Smax_lng = 2.253922
Smax_lat = 48.897599

In [3]:
# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
to_proxy_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857')
to_original_transformer = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')

# Create corners of rectangle to be transformed to a grid
sw = shapely.geometry.Point((2.225684,48.885438))
ne = shapely.geometry.Point((2.253922,48.897599))

stepsize = 100 # 5 km grid step size

# Project corners to target projection
transformed_sw = to_proxy_transformer.transform(sw.x, sw.y) # Transform NW point to 3857
transformed_ne = to_proxy_transformer.transform(ne.x, ne.y) # .. same for SE

# Iterate over 2D area
gridpoints = []
x = transformed_sw[0]
while x < transformed_ne[0]:
    y = transformed_sw[1]
    while y < transformed_ne[1]:
        p = shapely.geometry.Point(to_original_transformer.transform(x, y))
        gridpoints.append(p)
        y += stepsize
    x += stepsize

with open('cube.csv', 'w') as of:
    of.write('lonitude;latitude\n')
    for p in gridpoints:
        of.write('{:f};{:f}\n'.format(p.x, p.y))

In [4]:
LD_grid = pd.read_csv("cube.csv",engine='python',sep=',')
LD_grid.insert(0, 'Area(km2)', '1')
#print(LD_grid)